### 1. 스토리 데이터 불러오기
- ATHENA 프로그램을 통해 다운받기
- 데이터 내용 : 클릭된 모든 기사별 페이지뷰 
- 기간 : 7월
- 퍼블리셔 : 파트너스
- 카테고리 : CA0517

SELECT V2.nid, V2.visit, V2.newcategory, V2.rank					
FROM 					
   (SELECT V.*					
         , RANK() over (order by visit desc) as rank					
    FROM 					
       (SELECT nid, newcategory,count(*) as visit					
        FROM   elasticsearch_parquet					
        WHERE 1=1					
        AND  month = '07'					
        AND  type = 'VISIT'					
        AND  newcategory in ('CA0517')					
        AND  level in (60,61,65)					
        GROUP BY nid, type,newcategory					
        ) V					
    )V2

In [6]:
import pandas as pd
visit = pd.read_csv("스토리_p_visit_2207.csv")

In [7]:
visit.head()

,nid,visit,newcategory,rank
0,2022071002242909002,921562,CA0517,1
1,2022072817301576141,564737,CA0517,2
2,2022072914550299594,540323,CA0517,3
3,2022071112150188407,540165,CA0517,4
4,2022072018040874544,539323,CA0517,5


In [8]:
visit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10793 entries, 0 to 10792
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   nid          10793 non-null  int64 
 1   visit        10793 non-null  int64 
 2   newcategory  10793 non-null  object
 3   rank         10793 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 337.4+ KB


In [9]:
# 컬럼 'nid' 문자형으로 바꾸기
visit['nid']= visit['nid'].astype('str')

### 2. 클릭된 콘텐츠 키워드 데이터 가져오기

* 콘텐츠 데이터 : 6월~7월

In [13]:
# 기사 제목,nid 가져오기
import pymysql
connect = pymysql.connect(host='ec2-3-35-194-202.ap-northeast-2.compute.amazonaws.com'
                        , port = 3306
                        , user='newspic_read'
                        , password ='shxlvmffjtm1!'
                        , db = 'newspic'
                        , charset='utf8')
# connection으로부터 cursor 생성
cur = connect.cursor()
# 수집 뉴스 테이블 picNews 가져오기
query_news = '''
SELECT pn.author, pn.category, tc.category as newcategory, nc.codeName,nc.pcode, TW.*
FROM  picNews pn inner join tbContentsInfo tc on pn.nid= tc.nid
                 left join newsCate nc        ON tc.category = nc.code   
                 left JOIN tbNewsTitleWord TW ON pn.nid = TW.nid   
WHERE 1=1 
AND   pn.nid >= '2022060100000099999' 
AND   pn.nid <= '2022073123595999999'
AND   tc.category ='CA0517'
ORDER BY pn.pubDate DESC, pn.nid DESC
            '''
df_news = pd.read_sql(query_news, connect)
df_news = df_news[['nid','title','pcode','codeName','author','word','text_rank_word']]

In [14]:
vis_news = pd.merge(visit, df_news, on='nid')
vis_news.head()

,nid,visit,newcategory,rank,title,pcode,codeName,author,word,text_rank_word
0,2022071002242909002,921562,CA0517,1,여자 고객 타투 시술중 사고,CA05,스토리,레몬판,여자 고객 문신 시술 장애,여자 고객 문신 시술 장애
1,2022072817301576141,564737,CA0517,2,어제 파혼했다,CA05,스토리,유머대장,파혼,먹먹 파혼
2,2022072914550299594,540323,CA0517,3,한 여자의 살면서 가장 소름돋았던 순간,CA05,스토리,시보드,여자 소름 순간,친구 여자 핸드폰 문자 장소
3,2022071112150188407,540165,CA0517,4,여자얼굴로 보는 나의 mbti취향,CA05,스토리,시보드,여자 화상 취향,취향 고른 진짜 고르고 게임링크
4,2022072018040874544,539323,CA0517,5,모텔 앞에서 본 커플을 잊을 수 없다.ssul,CA05,스토리,담다유머,모텔 커플,모텔 커플


In [15]:
vis_news['date'] = vis_news['nid'].str[:8]

In [16]:
# 모든 행을 보고자할때 
vis_news.groupby('date').sum()

,visit,rank
date,,
20220601,2231,327791
20220602,2364,485237
20220603,2449,383898
20220604,1714,356054
20220605,1782,630435
...,...,...
20220727,1275720,439669
20220728,2127910,434930
20220729,1561473,461460


In [17]:
vis_news.head()

,nid,visit,newcategory,rank,title,pcode,codeName,author,word,text_rank_word,date
0,2022071002242909002,921562,CA0517,1,여자 고객 타투 시술중 사고,CA05,스토리,레몬판,여자 고객 문신 시술 장애,여자 고객 문신 시술 장애,20220710
1,2022072817301576141,564737,CA0517,2,어제 파혼했다,CA05,스토리,유머대장,파혼,먹먹 파혼,20220728
2,2022072914550299594,540323,CA0517,3,한 여자의 살면서 가장 소름돋았던 순간,CA05,스토리,시보드,여자 소름 순간,친구 여자 핸드폰 문자 장소,20220729
3,2022071112150188407,540165,CA0517,4,여자얼굴로 보는 나의 mbti취향,CA05,스토리,시보드,여자 화상 취향,취향 고른 진짜 고르고 게임링크,20220711
4,2022072018040874544,539323,CA0517,5,모텔 앞에서 본 커플을 잊을 수 없다.ssul,CA05,스토리,담다유머,모텔 커플,모텔 커플,20220720


In [18]:
# 키워드 행번호에 따라서 행렬화
wordlist = vis_news['word'].str.split(" ",expand=True)
wordlist2 = wordlist.stack().reset_index(level=1, drop=True).to_frame('keyword')

In [19]:
vis_news2 = vis_news.merge(wordlist2, left_index=True, right_index=True, how='left')

In [20]:
vis_news2.head()

,nid,visit,newcategory,rank,title,pcode,codeName,author,word,text_rank_word,date,keyword
0,2022071002242909002,921562,CA0517,1,여자 고객 타투 시술중 사고,CA05,스토리,레몬판,여자 고객 문신 시술 장애,여자 고객 문신 시술 장애,20220710,여자
0,2022071002242909002,921562,CA0517,1,여자 고객 타투 시술중 사고,CA05,스토리,레몬판,여자 고객 문신 시술 장애,여자 고객 문신 시술 장애,20220710,고객
0,2022071002242909002,921562,CA0517,1,여자 고객 타투 시술중 사고,CA05,스토리,레몬판,여자 고객 문신 시술 장애,여자 고객 문신 시술 장애,20220710,문신
0,2022071002242909002,921562,CA0517,1,여자 고객 타투 시술중 사고,CA05,스토리,레몬판,여자 고객 문신 시술 장애,여자 고객 문신 시술 장애,20220710,시술
0,2022071002242909002,921562,CA0517,1,여자 고객 타투 시술중 사고,CA05,스토리,레몬판,여자 고객 문신 시술 장애,여자 고객 문신 시술 장애,20220710,장애


In [21]:
vis_news2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24616 entries, 0 to 6476
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   nid             24616 non-null  object
 1   visit           24616 non-null  int64 
 2   newcategory     24616 non-null  object
 3   rank            24616 non-null  int64 
 4   title           24616 non-null  object
 5   pcode           24616 non-null  object
 6   codeName        24616 non-null  object
 7   author          24616 non-null  object
 8   word            24616 non-null  object
 9   text_rank_word  24616 non-null  object
 10  date            24616 non-null  object
 11  keyword         24616 non-null  object
dtypes: int64(2), object(10)
memory usage: 2.4+ MB


In [51]:
vis_news2.to_csv("파트너스_스토리기사리스트_2207.csv", encoding='utf-8-sig')

#### 피봇1. 키워드 포함된 콘텐츠 페이지뷰

In [27]:
import numpy as np
# visit : 키워드 포함된 콘텐츠 페이지뷰
v_pivot = pd.pivot_table(vis_news2 , index='keyword',values='visit',aggfunc=np.sum).sort_values(by='visit', ascending=False)

In [28]:
v_pivot.head(50)

,visit
keyword,
여자,15185407
JPG,8928035
남자,6979580
남자친구,5241037
친구,4686592
여자친구,4555391
결혼,4187464
남편,3686248
이유,2370807


In [29]:
v_pivot = v_pivot.reset_index()

#### 피봇2. 키워드 포함된 클릭된 콘텐츠 개수

In [30]:
# nid : 키워드 포함된 클릭된 콘텐츠 개수
n_pivot = vis_news2.groupby('keyword').count().sort_values(by='nid', ascending=False)

In [31]:
n_pivot =n_pivot[['nid']]

In [32]:
n_pivot.head()

,nid
keyword,
여자,1514
JPG,1187
남자,1164
결혼,525
여자친구,495


In [33]:
n_pivot = n_pivot.reset_index()

### 3. 콘텐츠 키워드별 수집수 가져오기

* 콘텐츠 데이터 : 7월

In [35]:
# 기사 제목,nid 가져오기
import pymysql
connect = pymysql.connect(host='ec2-3-35-194-202.ap-northeast-2.compute.amazonaws.com'
                        , port = 3306
                        , user='newspic_read'
                        , password ='shxlvmffjtm1!'
                        , db = 'newspic'
                        , charset='utf8')
# connection으로부터 cursor 생성
cur = connect.cursor()
# 수집 뉴스 테이블 picNews 가져오기
query2 = '''
SELECT pn.author, pn.category, tc.category as newcategory, nc.codeName,nc.pcode, TW.*
FROM  picNews pn inner join tbContentsInfo tc on pn.nid= tc.nid
                 left join newsCate nc        ON tc.category = nc.code   
                 left JOIN tbNewsTitleWord TW ON pn.nid = TW.nid   
WHERE 1=1 
AND   pn.nid >= '2022070100000099999' 
AND   pn.nid <= '2022073123595999999'
AND   tc.category ='CA0517'
ORDER BY pn.pubDate DESC, pn.nid DESC
            '''
news = pd.read_sql(query2, connect)
news = news[['nid','title','pcode','codeName','author','word','text_rank_word']]

####  피봇3.  키워드별 수집수 컬럼 생성

In [45]:
news.head()

,nid,title,pcode,codeName,author,word,text_rank_word
0,None,None,CA05,스토리,이슈야,None,None
1,None,None,CA05,스토리,이슈야,None,None
2,None,None,CA05,스토리,이슈야,None,None
3,None,None,CA05,스토리,이슈야,None,None
4,2022073123355985740,해외에서 게이커플 의혹 나왔던 프로게이머 둘,CA05,스토리,레몬판,해외 커플 의혹 프로게이머,박영서 브래디 어질리티 지라디 카리브


In [37]:
news.count()

nid               5612
title             5612
pcode             5703
codeName          5703
author            5703
word              5612
text_rank_word    5612
dtype: int64

In [38]:
# 키워드 행번호에 따라서 행렬화
n_wordlist = news['word'].str.split(" ",expand=True)
n_wordlist2 = n_wordlist.stack().reset_index(level=1, drop=True).to_frame('keyword')

In [39]:
n_wordlist2

,keyword
4,해외
4,커플
4,의혹
4,프로게이머
5,비키니
...,...
5702,남녀
5702,결혼
5702,나이
5702,마지노선


In [40]:
news2 = df_news.merge(n_wordlist2, left_index=True, right_index=True, how='left')

In [41]:
# 키워드에 따른 수집수컬럼 구하기
nid_pivot = news2.groupby('keyword').count().sort_values(by='nid', ascending=False)

In [42]:
nid_pivot =nid_pivot[['nid']]
nid_pivot = nid_pivot.reset_index()

In [43]:
nid_pivot

,keyword,nid
0,여자,1236
1,JPG,1096
2,남자,1050
3,결혼,477
4,친구,409
...,...,...
3446,인생네컷,1
3447,개소리,1
3448,인사,1
3449,복부,1


In [44]:
news2[news2['keyword']=='근황']

,nid,title,pcode,codeName,author,word,text_rank_word,keyword
6,2022073123250010023,2022년 7월 야가다 월급 근황,CA05,스토리,이슈야,야가다 월급 근황,조공 기공 업종 공포설 단가,근황
27,2022073120200959841,출산율 반전 못하는 암울한 일본 근황.jpg,CA05,스토리,움짤저장소,출산 반전 암울 일본 근황 JPG,ㅎㅎ 출산 반전 암울 일본,근황
42,2022073119250185383,2022년 7월 야가다 월급 근황....jpg,CA05,스토리,시보드,야가다 월급 근황 JPG,조공 기공 업종 공포설 단가,근황
58,2022073118205467840,출산율 반전 못하는 암울한 일본 근황.jpg,CA05,스토리,엠봉,출산 반전 암울 일본 근황 JPG,일본 ㅎㅎ 출산 반전 암울,근황
79,2022073116450262246,출산율 반전 못하는 암울한 일본 근황.jpg,CA05,스토리,시보드,출산 반전 암울 일본 근황 JPG,일본 ㅎㅎ 출산 반전 암울,근황
...,...,...,...,...,...,...,...,...
5574,2022070116585160793,셋째 임신했던 장영란 근황...JPG,CA05,스토리,레몬판,임신 장영란 근황 JPG,길거리 처음본 누님 유산 기운,근황
5582,2022070116450204939,셋째 임신했던 장영란 근황...JPG,CA05,스토리,이슈야,임신 장영란 근황 JPG,길거리 처음본 누님 유산 기운,근황
5607,2022070114283871119,셋째 임신했던 장영란 근황,CA05,스토리,사이다판,임신 장영란 근황,길거리 처음본 누님 유산 기운,근황
5635,2022070111274347961,임신했다던 장영란 안타까운 근황..,CA05,스토리,이슈야,임신 장영 근황,유산 임신 장영 근황,근황


### 3. 피봇 3개 합치기

In [46]:
f_pivot =pd.merge(v_pivot, n_pivot,on='keyword')

In [47]:
f_pivot

,keyword,visit,nid
0,여자,15185407,1514
1,JPG,8928035,1187
2,남자,6979580,1164
3,남자친구,5241037,470
4,친구,4686592,459
...,...,...,...
4305,당제,1,1
4306,핀란드,1,1
4307,의무,1,1
4308,틀니,1,1


In [48]:
f2_pivot =pd.merge(f_pivot, nid_pivot, on='keyword')

In [49]:
f2_pivot.head()

,keyword,visit,nid_x,nid_y
0,여자,15185407,1514,1236
1,JPG,8928035,1187,1096
2,남자,6979580,1164,1050
3,남자친구,5241037,470,366
4,친구,4686592,459,409


In [50]:
f2_pivot.to_csv("파트너스_스토리_2207.csv", encoding='utf-8-sig')